In [1]:
import torch
import numpy as np
import random
import torch.optim as optim

from algorithms import *
from models import *
from dataloaders import *

from tabulate import tabulate

In [2]:
device = 'cpu'
dataset = GermanDataset(device=device)

In [3]:
def lr_kde_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(int(seeds[i]))
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = MLP(num_features=dataset.XZ_train.shape[1],n_layers=[4])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_kde_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            h=hp['h'], 
            delta_huber=hp['delta_huber'], 
            optimal_effort=False, 
            delta_effort=hp['delta_effort'],
            effort_iter=hp['effort_iter'],
            effort_lr=hp['effort_lr']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test


def experiment_runner(dataset, SGD_hp, EI_hp, DP_hp, EO_hp, EODD_hp, seeds):
    
    _, _, SGD = lr_kde_model_runner(dataset, SGD_hp, seeds)
    _, _, EI = lr_kde_model_runner(dataset, EI_hp, seeds)
    _, _, DP = lr_kde_model_runner(dataset, DP_hp, seeds)
    _, _, EO = lr_kde_model_runner(dataset, EO_hp, seeds)
    _, _, EODD = lr_kde_model_runner(dataset, EODD_hp, seeds)
    
    return SGD, EI, DP, EO, EODD

def hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_kde_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

#### SGD Hyperparameter Selection

In [4]:
SGD_hp_test = {}
SGD_hp_test['learning_rate'] = [0.0001, 0.001, 0.01, 0.1]
SGD_hp_test['lambda_'] = [0]
SGD_hp_test['n_epochs'] = 100
SGD_hp_test['batch_size'] = 256
SGD_hp_test['fairness'] = ''
SGD_hp_test['h'] = 0.01
SGD_hp_test['delta_huber'] = 0.5
SGD_hp_test['delta_effort'] = 1.1
SGD_hp_test['effort_iter'] = 20
SGD_hp_test['effort_lr'] = 15

hyperparameter_test(dataset, SGD_hp_test, seed=1)

training seed 1 started


Training: 100%|██████████| 100/100 [00:01<00:00, 73.54epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:01<00:00, 87.86epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:01<00:00, 86.50epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:01<00:00, 89.22epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei         dp         eo       eodd
---------------  ---------  ----------------  --------------  ----------  ---------  ---------  ---------
         0.0001          0          0.696875         0.71875  0           0          0          0
         0.001           0          0.764062         0.775    0.00333333  0.0407197  0.0440711  0.0440711
         0.01            0          0.785937         0.7875   0.00645161  0.0486742  0.0494071  0.0494071
         0.1             0          0.796875         0.75     0.106667    0.0861742  0.0721344  0.0757576


In [5]:
a = 0
b = 0
c = 0
try:
    ei_z = abs(a/(a + b) - c)
except ZeroDivisionError:
    if a == 0: 
        ei_z = 0
    else:
        ei_z = 1

The learning rate is decided as 0.01.

In [6]:
SGD_hp = SGD_hp_test.copy()
SGD_hp['learning_rate'] = 0.01
SGD_hp['lambda_'] = 0


#### EI Hyperparameter Test

In [7]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.01]
EI_hp_test['lambda_'] = [0, 0.2, 0.4, 0.6, 0.8]
EI_hp_test['fairness'] = 'EI'

hyperparameter_test(dataset, EI_hp_test, seed=1)

training seed 1 started


Training: 100%|██████████| 100/100 [00:02<00:00, 41.64epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:02<00:00, 38.40epochs/s]


Training finished for all seeds.
training seed 1 started


...

Training finished for all seeds.
training seed 1 started


Training:   7%|▋         | 7/100 [00:00<00:01, 63.08epochs/s]/opt/homebrew/Caskroom/miniforge/base/envs/sampler_fairness/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/Caskroom/miniforge/base/envs/sampler_fairness/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
Training: 100%|██████████| 100/100 [00:02<00:00, 39.91epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:02<00:00, 43.19epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei         dp         eo       eodd
---------------  ---------  ----------------  --------------  ----------  ---------  ---------  ---------
           0.01        0            0.790625         0.7875   0.00738916  0.0308712  0.0353755  0.0353755
           0.01        0.2          0.785937         0.78125  0.0294118   0.0450758  0.0547431  0.0547431
           0.01        0.4          0.782813         0.75625  0.04375     0.0575758  0.0687747  0.0687747
           0.01        0.6          0.773438         0.74375  0.0270833   0.0272727  0.0600791  0.0818182
           0.01        0.8          0.76875          0.75     0.00168067  0.0539773  0.0968379  0.0968379


The lambda for EI is decided as 0.8.

In [8]:
EI_hp = EI_hp_test.copy()
EI_hp['learning_rate'] = 0.01
EI_hp['lambda_'] = 0.8

#### DP Hyperparameter Selection   

In [9]:
DP_hp_test = SGD_hp_test.copy()
DP_hp_test['learning_rate'] = [0.01]
DP_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
DP_hp_test['fairness'] = 'DP'

hyperparameter_test(dataset, DP_hp_test, seed=1)

training seed 1 started


Training: 100%|██████████| 100/100 [00:01<00:00, 66.69epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:01<00:00, 68.65epochs/s]


Training finished for all seeds.
training seed 1 started


...

Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:01<00:00, 64.94epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:01<00:00, 75.12epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei           dp          eo       eodd
---------------  ---------  ----------------  --------------  ----------  -----------  ----------  ---------
           0.01       0             0.785937         0.7875   0.00645161  0.0486742    0.0494071   0.0494071
           0.01       0.2           0.784375         0.78125  0.0357143   0.0371212    0.0353755   0.0353755
           0.01       0.6           0.773438         0.7625   0.00865801  0.00587121   0.0233202   0.0808081
           0.01       0.8           0.778125         0.7625   0.0229885   0.000568182  0.0179842   0.0818182
           0.01       0.9           0.767188         0.75625  0.0142857   0.0378788    0.0114625   0.147475
           0.01       0.95          0.759375         0.75     0.117647    0.0153409    0.00671937  0.0626263


The lambda for DP is decided as 0.6.

In [10]:
DP_hp = DP_hp_test.copy()
DP_hp['learning_rate'] = 0.01
DP_hp['lambda_'] = 0.6

#### EO Hyperparameter Selection

In [11]:
EO_hp_test = SGD_hp_test.copy()
EO_hp_test['learning_rate'] = [0.01]
EO_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
EO_hp_test['fairness'] = 'EO'

hyperparameter_test(dataset, EO_hp_test, seed=1)

training seed 1 started


Training: 100%|██████████| 100/100 [00:01<00:00, 75.81epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:01<00:00, 77.79epochs/s]


Training finished for all seeds.
training seed 1 started


...

Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:01<00:00, 77.85epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:01<00:00, 77.15epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei         dp         eo       eodd
---------------  ---------  ----------------  --------------  ----------  ---------  ---------  ---------
           0.01       0             0.785937         0.7875   0.00645161  0.0486742  0.0494071  0.0494071
           0.01       0.2           0.792188         0.78125  0.0125      0.0575758  0.0634387  0.0634387
           0.01       0.6           0.785937         0.7625   0.0667727   0.0263258  0.0513834  0.079798
           0.01       0.8           0.767188         0.76875  0.0416667   0.0174242  0.0373518  0.079798
           0.01       0.9           0.767188         0.75625  0.0428571   0.0189394  0.015415   0.0616162
           0.01       0.95          0.76875          0.76875  0.115385    0.0109848  0.0353755  0.150505


The lambda for EO is decided as 0.9.

In [12]:
EO_hp = EO_hp_test.copy()
EO_hp['learning_rate'] = 0.01
EO_hp['lambda_'] = 0.9

#### EODD Hyeprparameter Selection

In [13]:
EODD_hp_test = SGD_hp_test.copy()
EODD_hp_test['learning_rate'] = [0.01]
EODD_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
EODD_hp_test['fairness'] = 'EODD'

hyperparameter_test(dataset, EODD_hp_test, seed=1)

training seed 1 started


Training: 100%|██████████| 100/100 [00:01<00:00, 67.07epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:01<00:00, 66.67epochs/s]


Training finished for all seeds.
training seed 1 started


...

Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:01<00:00, 65.88epochs/s]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [00:01<00:00, 64.03epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei          dp         eo       eodd
---------------  ---------  ----------------  --------------  ----------  ----------  ---------  ---------
           0.01       0             0.785937         0.7875   0.00645161  0.0486742   0.0494071  0.0494071
           0.01       0.2           0.795312         0.775    0.0599078   0.0638258   0.0634387  0.0634387
           0.01       0.6           0.776563         0.74375  0.075       0.00757576  0.0252964  0.125253
           0.01       0.8           0.764062         0.75     0.111111    0.0172348   0.0407115  0.173737
           0.01       0.9           0.698438         0.725    1           0.00625     0          0.0222222
           0.01       0.95          0.696875         0.71875  0           0           0          0


The lambda for EODD is picked as 0.2.

In [14]:
EODD_hp = EODD_hp_test.copy()
EODD_hp['learning_rate'] = 0.01
EODD_hp['lambda_'] = 0.2

#### Model training

In [15]:
seeds = np.arange(1,6)

SGD, EI, DP, EO, EODD = experiment_runner(dataset, SGD_hp, EI_hp, DP_hp, EO_hp, EODD_hp, seeds)

/var/folders/mt/mgpv1mld71dfr_kn_fq57str0000gn/T/ipykernel_18310/415546996.py:29: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(seeds[i])


training seed 1 started


Training: 100%|██████████| 100/100 [00:01<00:00, 90.07epochs/s]


training seed 2 started


Training: 100%|██████████| 100/100 [00:01<00:00, 90.21epochs/s]


...

training seed 4 started


Training: 100%|██████████| 100/100 [00:01<00:00, 64.50epochs/s]


training seed 5 started


Training: 100%|██████████| 100/100 [00:01<00:00, 67.88epochs/s]

Training finished for all seeds.


In [16]:
result = []
models = ["SGD", "EI", "DP", "EO", "EODD"]
sol = [SGD, EI, DP, EO, EODD]
for i in range(len(models)):
    c = []
    c.append(models[i])
    res = sol[i]
    c.append(res['accuracy_mean'])
    c.append(res['accuracy_var'])
    c.append(res['ei_mean'])
    c.append(res['ei_var'])
    c.append(res['dp_mean'])
    c.append(res['dp_var'])
    c.append(res['eo_mean'])
    c.append(res['eo_var'])
    c.append(res['eodd_mean'])
    c.append(res['eodd_var'])
    result.append(c)

print(tabulate(result, headers=["model","accuracy_mean","accuracy_var","ei_mean","ei_var", "dp_mean", "dp_var","eo_mean","eo_var","eodd_mean","eodd_var"]))

model      accuracy_mean    accuracy_var    ei_mean     ei_var    dp_mean     dp_var    eo_mean      eo_var    eodd_mean    eodd_var
-------  ---------------  --------------  ---------  ---------  ---------  ---------  ---------  ----------  -----------  ----------
SGD                0.779       0.0106771  0.0586691  0.0456625  0.0682716  0.0133305  0.0586399  0.0110849     0.0632305   0.0120873
EI                 0.748       0.0328024  0.0101478  0.0189926  0.0536914  0.0254791  0.050957   0.0219105     0.0532481   0.0227648
DP                 0.776       0.0146287  0.113786   0.0223065  0.0349506  0.0116383  0.0414008  0.00474043    0.0485858   0.0150342
EO                 0.763       0.0282135  0.0981386  0.04767    0.0197037  0.0149372  0.0220171  0.0148094     0.0709756   0.0397984
EODD               0.782       0.012083   0.0631054  0.0293485  0.0623333  0.0107852  0.0525316  0.0119913     0.0587019   0.0130962
